In [1]:
import os
from torch import nn
import torch, transformers
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pandas as pd
import numpy as np
from sklearn import metrics
import re,pickle,random,json,os
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,Dataset
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm
import math 
import sklearn
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 10)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids).values()
        #print(output_1)
        #print(_)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BERTClass()
device = 'cuda'
model.to(device)
model = torch.load('/home/wooseok/llama2-chat/cdlilogue10_bert_pts/epoch18_symptom10_classifier_model.pt')

model.eval()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/wooseok/miniconda3/envs/mh/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so...


/home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
2023-12-13 02:57:00.438592: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 02:57:01.713960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [6]:
#max_len = 300
#batch_size = 16

class CustomDataset(Dataset):
    def __init__(self, df,tokenizer, max_len = 512):
        self.tokenizer = tokenizer
        self.data = df
        self.utter = df['hypothesis']
        self.targets = self.data.label
        self.max_len = max_len
    def __len__(self):
        return len(self.targets)
    def __getitem__(self, index):
        utter = self.utter[index]
        inputs = self.tokenizer.encode_plus(
            utter,
            None,
            add_special_tokens = True,
            max_length = self.max_len,
            padding = 'max_length',
            return_token_type_ids = True,
            truncation = True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }    
        

In [7]:
import json

with open('acdp/llama7b/generation_results.json') as f:
    acdp_7b = json.load(f)
with open('acdp/llama13b/generation_results.json') as f:
    acdp_13b = json.load(f)
with open('ours/llama7b/generation_results.json') as f:
    ours_7b = json.load(f)
with open('ours/llama13b/generation_results.json') as f:
    ours_13b = json.load(f)

with open("/home/wooseok/llama2-chat/llama2-chat-generated/result_generated_llama_chat.pkl","rb") as f:
     prompt_chat= pickle.load(f)
prompt_chat_7b = prompt_chat[['generated_utterance','attributes']]
prompt_chat_7b.columns = ['hypothesis','symptom']    

with open("/home/wooseok/llama2-chat/llama2-generated/result_generated_llama.pkl","rb") as f:
     prompt= pickle.load(f)
prompt_7b = prompt[['generated_utterance','attributes']]
prompt_7b.columns = ['hypothesis','symptom']

with open("/home/wooseok/llama2-chat/llama213-chat-generated/result_generated_llama_chat13.pkl","rb") as f:
     prompt_chat13= pickle.load(f)
prompt_chat_13b = prompt_chat13[['generated_utterance','attributes']]
prompt_chat_13b.columns = ['hypothesis','symptom']    

with open("/home/wooseok/llama2-chat/llama213-generated/result_generated_llama13.pkl","rb") as f:
     prompt13= pickle.load(f)
prompt_13b = prompt13[['generated_utterance','attributes']]
prompt_13b.columns = ['hypothesis','symptom']




def accuracy(data,prompt, without0, mercy,average):
    
    if prompt == False:
        data = pd.DataFrame(data['data'])[['hypothesis','symptom']]
    data['label']= [np.array([0] * 10) for _ in range(len(data))]
    
    Symptom_list = ['cough', 'fever', 'sore throat', 'breathing problem', 'headache', 'pain', 'virus' , 'eye problem' , 'allergy','pneumonia']

    def set_label(row):
        label = [1 if symptom in row['symptom'] else 0 for symptom in Symptom_list]
        return label

    # apply 함수를 사용하여 label 값을 설정
    #print(data.columns)
    data['label'] = data.apply(set_label, axis=1)
    #del data['symptom']

    data= data.reset_index(drop=True)
    
    if without0 == True:
    
        for i in range(len(data)):
            if np.sum(data['label'][i])==0 : 
                data.drop(i,inplace = True)
        data= data.reset_index(drop=True)

    
    if without0 ==False:
        for i in range(len(data)):
            if data['hypothesis'][i] == '':
                data.drop(i,inplace = True)
        data= data.reset_index(drop=True)

    #print(len(data))
    test_set = CustomDataset(data, tokenizer, 256)
    test_loader = DataLoader(test_set,batch_size=20, shuffle = False)



    val_targets=[]
    val_outputs=[]

    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

        val_preds = (np.array(val_outputs) > 0.5).astype(int)
        
        if mercy == True:
            val_targets = np.array(val_targets).astype(int)
            acc =0
            for i in range(len(val_preds)):
                if np.all(val_preds[i][np.where(val_targets[i]==1)]==1) == True: 
                    acc+=1
            #print("------------------------------",acc)
            accc = acc/len(val_preds)
            #print(accc)
            
        if mercy == False:
            EMR = round(sklearn.metrics.accuracy_score(np.array(val_targets), val_preds, normalize=True, sample_weight=None),3)
            hl =round(sklearn.metrics.hamming_loss(np.array(val_targets), val_preds),3)
            precision= round(sklearn.metrics.precision_score(y_true=np.array(val_targets), y_pred=val_preds, average =average),3)
            recall = round(sklearn.metrics.recall_score(y_true=np.array(val_targets), y_pred=val_preds, average =average),3)
            f1 = round(sklearn.metrics.f1_score(y_true=np.array(val_targets), y_pred=val_preds, average =average),3)
            
            #acc
            val_targets = np.array(val_targets).astype(int)
            acc =0
            for i in range(len(val_preds)):
                if np.all(val_preds[i][np.where(val_targets[i]==1)]==1) == True: 
                    acc+=1
            #print("------------------------------",acc)
            accc = acc/len(val_preds)
            #print('Exact Match Ratio: {0}'.format(EMR)) 
            #print('Precision: {0}'.format(precision))
            #print('Recall: {0}'.format(recall)) 
            #print('F1 Measure: {0}'.format(f1))
            #print('Hamming loss: {0}'.format(hl))

            return EMR,hl,precision,recall,f1,accc
def per_tab(average):
    a=accuracy(ours_7b,prompt = False,without0 = False, mercy = False,average =average )
    b=accuracy(ours_13b,prompt = False,without0 = False, mercy = False,average =average )
    c=accuracy(prompt_chat_7b,prompt =True,without0 = False, mercy = False,average =average )
    d=accuracy(prompt_7b,prompt =True,without0 = False, mercy = False,average =average )
    e=accuracy(acdp_7b,prompt = False,without0 = False, mercy = False,average =average )
    f=accuracy(acdp_13b,prompt = False,without0 = False, mercy = False,average =average )
    g=accuracy(prompt_chat_13b,prompt =True,without0 = False, mercy = False,average =average )
    h=accuracy(prompt_13b,prompt =True,without0 = False, mercy = False,average =average )
    
    if average == 'micro':
        columns=[['EMR','Hamming loss','Precision(micro)','Recall(micro)','F1-score(micro)','accuracy']]
    elif average == 'macro':
        columns=[['EMR','Hamming loss','Precision(macro)','Recall(macro)','F1-score(macro)','accuracy']]
    elif average == 'weighted':
        columns=[['EMR','Hamming loss','Precision(weighted)','Recall(weighted)','F1-score(weighted)','accuracy']]
    else:
        columns=[['EMR','Hamming loss','Precision(samples)','Recall(samples)','F1-score(samples)','accuracy']]
        
    df = pd.DataFrame([list(a),list(b),list(c),list(d),list(e),list(f),list(g),list(h)],columns = columns,index = ['ours_7b','ours_13b','prompt_chat_7b','prompt_7b','acdp_7b', 'acdp_13b','prompt_chat_13b','prompt_13b'])
    #print(df)
    return df


In [8]:
import warnings
# 경고창 숨기기
warnings.filterwarnings('ignore') 
a=per_tab('micro')
print("a")
b=per_tab('macro')
print("b")
c=per_tab('weighted')
print("c")
d = per_tab('samples')
print("d")

a['Precision(macro)'] = b['Precision(macro)']
a['Recall(macro)'] = b['Recall(macro)']
a['F1-score(macro)'] = b['F1-score(macro)']

a['Precision(weighted)'] = c['Precision(weighted)']
a['Recall(weighted)'] = c['Recall(weighted)']
a['F1-score(weighted)'] = c['F1-score(weighted)']

a['Precision(samples)'] = d['Precision(samples)']
a['Recall(samples)'] = d['Recall(samples)']
a['F1-score(samples)'] = d['F1-score(samples)']

a = a.reindex(['ours_7b', 'ours_13b', 'acdp_7b', 'acdp_13b', 'prompt_7b','prompt_chat_7b','prompt_13b','prompt_chat_13b'])


df =a
def highlight_top2(col):
    is_top2 = col.isin(col.nlargest(1))
    return ['font-weight: bold' if v else '' for v in is_top2]

# Apply the style
styled_df = df.style.apply(highlight_top2)
styled_df




a
b
c
d


,"('EMR',)","('Hamming loss',)","('Precision(micro)',)","('Recall(micro)',)","('F1-score(micro)',)","('accuracy',)","('Precision(macro)',)","('Recall(macro)',)","('F1-score(macro)',)","('Precision(weighted)',)","('Recall(weighted)',)","('F1-score(weighted)',)","('Precision(samples)',)","('Recall(samples)',)","('F1-score(samples)',)"
ours_7b,0.851000,0.017000,0.768000,0.801000,0.784000,0.931429,0.718000,0.773000,0.733000,0.788000,0.801000,0.785000,0.228000,0.227000,0.223000
ours_13b,0.811000,0.021000,0.711000,0.794000,0.750000,0.922857,0.683000,0.771000,0.700000,0.776000,0.794000,0.764000,0.225000,0.225000,0.221000
acdp_7b,0.851000,0.018000,0.789000,0.743000,0.765000,0.914286,0.729000,0.718000,0.711000,0.820000,0.743000,0.769000,0.227000,0.215000,0.218000
acdp_13b,0.829000,0.020000,0.743000,0.743000,0.743000,0.908571,0.717000,0.733000,0.710000,0.763000,0.743000,0.742000,0.225000,0.212000,0.215000
prompt_7b,0.561000,0.071000,0.247000,0.426000,0.313000,0.815642,0.223000,0.363000,0.253000,0.283000,0.426000,0.318000,0.114000,0.112000,0.109000
prompt_chat_7b,0.542000,0.078000,0.274000,0.642000,0.384000,0.872222,0.327000,0.616000,0.402000,0.334000,0.642000,0.417000,0.150000,0.166000,0.152000
prompt_13b,0.567000,0.068000,0.247000,0.387000,0.301000,0.811111,0.267000,0.371000,0.292000,0.281000,0.387000,0.310000,0.110000,0.105000,0.104000
prompt_chat_13b,0.521000,0.087000,0.237000,0.584000,0.338000,0.872576,0.280000,0.528000,0.332000,0.317000,0.584000,0.381000,0.136000,0.162000,0.141000


In [26]:
def highlight_top2(col):
    is_top2 = col.isin(col.nlargest(1))
    return ['font-weight: bold' if v else '' for v in is_top2]

aaaaa = a[['accuracy','EMR','Hamming loss','Precision(micro)','Recall(micro)','F1-score(micro)','Precision(macro)','Recall(macro)','F1-score(macro)']]
df = aaaaa
df = df.reindex([ 'acdp_7b', 'acdp_13b', 'prompt_7b','prompt_chat_7b','prompt_13b','prompt_chat_13b','ours_7b', 'ours_13b'])

df
styled_df = df.style.format('{:.2%}').apply(highlight_top2)#.applymap(lambda x: 'font-weight: bold' if x in df.max(axis=0).nlargest(2).values else '')
#styled_df = df.style
styled_df


,"('accuracy',)","('EMR',)","('Hamming loss',)","('Precision(micro)',)","('Recall(micro)',)","('F1-score(micro)',)","('Precision(macro)',)","('Recall(macro)',)","('F1-score(macro)',)"
acdp_7b,91.43%,85.10%,1.80%,78.90%,74.30%,76.50%,72.90%,71.80%,71.10%
acdp_13b,90.86%,82.90%,2.00%,74.30%,74.30%,74.30%,71.70%,73.30%,71.00%
prompt_7b,81.56%,56.10%,7.10%,24.70%,42.60%,31.30%,22.30%,36.30%,25.30%
prompt_chat_7b,87.22%,54.20%,7.80%,27.40%,64.20%,38.40%,32.70%,61.60%,40.20%
prompt_13b,81.11%,56.70%,6.80%,24.70%,38.70%,30.10%,26.70%,37.10%,29.20%
prompt_chat_13b,87.26%,52.10%,8.70%,23.70%,58.40%,33.80%,28.00%,52.80%,33.20%
ours_7b,93.14%,85.10%,1.70%,76.80%,80.10%,78.40%,71.80%,77.30%,73.30%
ours_13b,92.29%,81.10%,2.10%,71.10%,79.40%,75.00%,68.30%,77.10%,70.00%


,accuracy,EMR,Hamming loss,Precision(micro),Recall(micro),F1-score(micro),Precision(macro),Recall(macro),F1-score(macro)
acdp_7b,0.914286,0.851,0.018,0.789,0.743,0.765,0.729,0.718,0.711
acdp_13b,0.908571,0.829,0.020,0.743,0.743,0.743,0.717,0.733,0.710
prompt_7b,0.815642,0.561,0.071,0.247,0.426,0.313,0.223,0.363,0.253
prompt_chat_7b,0.872222,0.542,0.078,0.274,0.642,0.384,0.327,0.616,0.402
prompt_13b,0.811111,0.567,0.068,0.247,0.387,0.301,0.267,0.371,0.292
prompt_chat_13b,0.872576,0.521,0.087,0.237,0.584,0.338,0.280,0.528,0.332
ours_7b,0.931429,0.851,0.017,0.768,0.801,0.784,0.718,0.773,0.733
ours_13b,0.922857,0.811,0.021,0.711,0.794,0.750,0.683,0.771,0.700


In [22]:
df.to_excel("output.xlsx") 

In [17]:
aaaaa

,accuracy,EMR,Hamming loss,Precision(micro),Recall(micro),F1-score(micro),Precision(macro),Recall(macro),F1-score(macro)
ours_7b,0.93,0.851,0.017,0.768,0.801,0.784,0.718,0.773,0.733
ours_13b,0.92,0.811,0.021,0.711,0.794,0.750,0.683,0.771,0.700
acdp_7b,0.91,0.851,0.018,0.789,0.743,0.765,0.729,0.718,0.711
acdp_13b,0.91,0.829,0.020,0.743,0.743,0.743,0.717,0.733,0.710
prompt_7b,0.82,0.561,0.071,0.247,0.426,0.313,0.223,0.363,0.253
prompt_chat_7b,0.87,0.542,0.078,0.274,0.642,0.384,0.327,0.616,0.402
prompt_13b,0.81,0.567,0.068,0.247,0.387,0.301,0.267,0.371,0.292
prompt_chat_13b,0.87,0.521,0.087,0.237,0.584,0.338,0.280,0.528,0.332


In [16]:
df = aaaaa
df['accuracy']=round(df['accuracy'],3)
df

,accuracy,EMR,Hamming loss,Precision(micro),Recall(micro),F1-score(micro),Precision(macro),Recall(macro),F1-score(macro)
ours_7b,0.93,0.851,0.017,0.768,0.801,0.784,0.718,0.773,0.733
ours_13b,0.92,0.811,0.021,0.711,0.794,0.750,0.683,0.771,0.700
acdp_7b,0.91,0.851,0.018,0.789,0.743,0.765,0.729,0.718,0.711
acdp_13b,0.91,0.829,0.020,0.743,0.743,0.743,0.717,0.733,0.710
prompt_7b,0.82,0.561,0.071,0.247,0.426,0.313,0.223,0.363,0.253
prompt_chat_7b,0.87,0.542,0.078,0.274,0.642,0.384,0.327,0.616,0.402
prompt_13b,0.81,0.567,0.068,0.247,0.387,0.301,0.267,0.371,0.292
prompt_chat_13b,0.87,0.521,0.087,0.237,0.584,0.338,0.280,0.528,0.332


In [54]:
mercy = True
average = 'micro'
data = prompt_13b
data['label']= [np.array([0] * 10) for _ in range(len(data))]

Symptom_list = ['cough', 'fever', 'sore throat', 'breathing problem', 'headache', 'pain', 'virus' , 'eye problem' , 'allergy','pneumonia']

def set_label(row):
    label = [1 if symptom in row['symptom'] else 0 for symptom in Symptom_list]
    return label

# apply 함수를 사용하여 label 값을 설정
#print(data.columns)
data['label'] = data.apply(set_label, axis=1)
#del data['symptom']


data= data.reset_index(drop=True)
for i in range(len(data)):
    if data['hypothesis'][i] == '':
        data.drop(i,inplace = True)
data= data.reset_index(drop=True)

data

,hypothesis,symptom,label
0,I've never had a headache and sore throat like...,"[headache, sore throat]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,How long is the waiting time to see the Doctor?,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Okay. How do you feel about a patient with th...,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,I have a sore throat.,[sore throat],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,Nope.,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...
355,I am a 17 year old male that was recently dia...,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
356,"I have been coughing a lot, and I have been h...",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
357,"What do I do, if I get fever, cough or chest ...",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
358,"No, I didn't",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [56]:
precision

0.247

In [57]:
recall

0.387

In [55]:

test_set = CustomDataset(data, tokenizer, 256)
test_loader = DataLoader(test_set,batch_size=20, shuffle = False)



val_targets=[]
val_outputs=[]

with torch.no_grad():
    for batch_idx, data in enumerate(test_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        val_targets.extend(targets.cpu().detach().numpy().tolist())
        val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    val_preds = (np.array(val_outputs) > 0.5).astype(int)

    if mercy == True:
        val_targets = np.array(val_targets).astype(int)
        acc =0
        for i in range(len(val_preds)):
            if np.all(val_preds[i][np.where(val_targets[i]==1)]==1) == True: 
                acc+=1
        #print("------------------------------",acc)
        accc = acc/len(val_preds)
        print(accc)
        

    if mercy == False:
        EMR = round(sklearn.metrics.accuracy_score(np.array(val_targets), val_preds, normalize=True, sample_weight=None),3)
        hl =round(sklearn.metrics.hamming_loss(np.array(val_targets), val_preds),3)
        precision= round(sklearn.metrics.precision_score(y_true=np.array(val_targets), y_pred=val_preds, average =average),3)
        recall = round(sklearn.metrics.recall_score(y_true=np.array(val_targets), y_pred=val_preds, average =average),3)
        f1 = round(sklearn.metrics.f1_score(y_true=np.array(val_targets), y_pred=val_preds, average =average),3)


0.8111111111111111


In [23]:
data = ours_13b
data = pd.DataFrame(data['data'])[['hypothesis','symptom']]
data['label']= [np.array([0] * 10) for _ in range(len(data))]

Symptom_list = ['cough', 'fever', 'sore throat', 'breathing problem', 'headache', 'pain', 'virus' , 'eye problem' , 'allergy','pneumonia']

def set_label(row):
    label = [1 if symptom in row['symptom'] else 0 for symptom in Symptom_list]
    return label

# apply 함수를 사용하여 label 값을 설정
#print(data.columns)
data['label'] = data.apply(set_label, axis=1)
#del data['symptom']


data= data.reset_index(drop=True)
for i in range(len(data)):
    if data['hypothesis'][i] == '':
        data.drop(i,inplace = True)
data= data.reset_index(drop=True)

data

,hypothesis,symptom,label
0,"Patient: I have a sore throat, ear ache, heada...","[headache, sore throat]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,Patient: Can you prescribe any medication?,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Patient: Thanks a lot for your time Doctor.,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"Patient: Hi Doctor, I have a sore throat. Can ...",[sore throat],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,Patient: I have no other symptoms. Should I ge...,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...
345,"Patient: Yes, I have completed my antibiotics.",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
346,Patient: I went to the doctors yesterday to ge...,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
347,Patient: Thanks a lot for your time Doctor.,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
348,"Patient: No, I did not get a chest X-ray done.",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [64]:
print(a.to_markdown())

| Model           |   EMR |   Hamming loss |   Precision(micro) |   Recall(micro) |   F1-score(micro) |   Precision(macro) |   Recall(macro) |   F1-score(macro) |   Precision(weighted) |   Recall(weighted) |   F1-score(weighted) |   Precision(samples) |   Recall(samples) |   F1-score(samples) |
|:----------------|------:|---------------:|-------------------:|----------------:|------------------:|-------------------:|----------------:|------------------:|----------------------:|-------------------:|---------------------:|---------------------:|------------------:|--------------------:|
| ours_7b         | 0.851 |          0.017 |              0.768 |           0.801 |             0.784 |              0.718 |           0.773 |             0.733 |                 0.788 |              0.801 |                0.785 |                0.228 |             0.227 |               0.223 |
| ours_13b        | 0.811 |          0.021 |              0.711 |           0.794 |             0.75  |         

In [160]:
a

,EMR,Hamming loss,Precision(micro),Recall(micro),F1-score(micro),Precision(macro),Recall(macro),F1-score(macro),Precision(weighted),Recall(weighted),F1-score(weighted),Precision(samples),Recall(samples),F1-score(samples)
ours_7b,0.851,0.017,0.768,0.801,0.784,0.718,0.773,0.733,0.788,0.801,0.785,0.228,0.227,0.223
ours_13b,0.811,0.021,0.711,0.794,0.750,0.683,0.771,0.700,0.776,0.794,0.764,0.225,0.225,0.221
prompt_chat_7b,0.542,0.078,0.275,0.642,0.385,0.330,0.616,0.404,0.336,0.642,0.418,0.151,0.166,0.152
prompt_7b,0.561,0.071,0.247,0.426,0.313,0.223,0.363,0.253,0.283,0.426,0.318,0.114,0.112,0.109
acdp_7b,0.851,0.018,0.789,0.743,0.765,0.729,0.718,0.711,0.820,0.743,0.769,0.227,0.215,0.218
acdp_13b,0.829,0.020,0.743,0.743,0.743,0.717,0.733,0.710,0.763,0.743,0.742,0.225,0.212,0.215


In [17]:
a

,EMR,Hamming loss,Precision(micro),Recall(micro),F1-score(micro),Precision(macro),Recall(macro),F1-score(macro),Precision(weighted),Recall(weighted),F1-score(weighted),Precision(samples),Recall(samples),F1-score(samples)
ours_7b,0.851,0.017,0.768,0.801,0.784,0.718,0.773,0.733,0.788,0.801,0.785,0.228,0.227,0.223
ours_13b,0.811,0.021,0.711,0.794,0.750,0.683,0.771,0.700,0.776,0.794,0.764,0.225,0.225,0.221
prompt_chat_7b,0.542,0.078,0.275,0.642,0.385,0.330,0.616,0.404,0.336,0.642,0.418,0.151,0.166,0.152
prompt_7b,0.561,0.071,0.247,0.426,0.313,0.223,0.363,0.253,0.283,0.426,0.318,0.114,0.112,0.109
acdp_7b,0.851,0.018,0.789,0.743,0.765,0.729,0.718,0.711,0.820,0.743,0.769,0.227,0.215,0.218
acdp_13b,0.829,0.020,0.743,0.743,0.743,0.717,0.733,0.710,0.763,0.743,0.742,0.225,0.212,0.215
prompt_chat_13b,0.521,0.087,0.237,0.584,0.338,0.280,0.528,0.332,0.317,0.584,0.381,0.136,0.162,0.141
prompt_13b,0.567,0.068,0.247,0.387,0.301,0.267,0.371,0.292,0.281,0.387,0.310,0.110,0.105,0.104


In [69]:
import pandas as pd

# Sample DataFrame
data = {
    "Model": ["ours_7b", "ours_13b", "acdp_7b", "acdp_13b", "prompt_7b", "prompt_chat_7b", "prompt_13b", "prompt_chat_13b"],
    "EMR": [0.851, 0.811, 0.851, 0.829, 0.561, 0.542, 0.567, 0.521],
    "Hamming loss": [0.017, 0.021, 0.018, 0.020, 0.071, 0.078, 0.068, 0.087],
}

df = pd.DataFrame(data)
df = df.set_index('Model')  # Assuming 'Model' is the index
df =a 
# Function to apply bold to the top two values
def highlight_top2(col):
    is_top2 = col.isin(col.nlargest(2))
    return ['font-weight: bold' if v else '' for v in is_top2]

# Apply the style
styled_df = df.style.apply(highlight_top2)
styled_df

,EMR,Hamming loss,Precision(micro),Recall(micro),F1-score(micro),Precision(macro),Recall(macro),F1-score(macro),Precision(weighted),Recall(weighted),F1-score(weighted),Precision(samples),Recall(samples),F1-score(samples)
Model,,,,,,,,,,,,,,
ours_7b,0.851000,0.017000,0.768000,0.801000,0.784000,0.718000,0.773000,0.733000,0.788000,0.801000,0.785000,0.228000,0.227000,0.223000
ours_13b,0.811000,0.021000,0.711000,0.794000,0.750000,0.683000,0.771000,0.700000,0.776000,0.794000,0.764000,0.225000,0.225000,0.221000
acdp_7b,0.851000,0.018000,0.789000,0.743000,0.765000,0.729000,0.718000,0.711000,0.820000,0.743000,0.769000,0.227000,0.215000,0.218000
acdp_13b,0.829000,0.020000,0.743000,0.743000,0.743000,0.717000,0.733000,0.710000,0.763000,0.743000,0.742000,0.225000,0.212000,0.215000
prompt_7b,0.561000,0.071000,0.247000,0.426000,0.313000,0.223000,0.363000,0.253000,0.283000,0.426000,0.318000,0.114000,0.112000,0.109000
prompt_chat_7b,0.542000,0.078000,0.275000,0.642000,0.385000,0.330000,0.616000,0.404000,0.336000,0.642000,0.418000,0.151000,0.166000,0.152000
prompt_13b,0.567000,0.068000,0.247000,0.387000,0.301000,0.267000,0.371000,0.292000,0.281000,0.387000,0.310000,0.110000,0.105000,0.104000
prompt_chat_13b,0.521000,0.087000,0.237000,0.584000,0.338000,0.280000,0.528000,0.332000,0.317000,0.584000,0.381000,0.136000,0.162000,0.141000


In [ ]:
print(.to_markdown())

AttributeError: 'Styler' object has no attribute 'to_markdown'

In [ ]:
93.14%
92.29% 91.43% 90.86%81.56%

| Method | Model | SA(관대) | SA_without0(관대) | SA(엄격) | SA_without0(엄격) | 
|---|---|---|---|---|---|
| ours (재우) | llama-2-7b |93.14%        |75.76%   |85.14%|66.67%|
| ours (재우) | llama-2-13b |92.29%       |72.73    |81.14%|60.61%|
| prompt (우석) | llama-2-7b-chat |87.22% |50.94%    |54.17%|33.96%|
| prompt (우석) | llama-2-7b |81.56%      |31.13%   |56.15%|25.47%|
| ACDP (준영) | llama-2-7b |91.43%        |69.70%    |85.14%|67.68%|
| ACDP (준영) | llama-2-13b |90.86%       |67.68%    |82.86%|64.65%|

- SA(관대)
  - 360개의 문장 테스트 
  - 밑과 같은 상황은 정답이라고 봄.(target 증상을 포함하니까)
  - target label : [0,0,1]
  - pred lable : [0,1,1]
- SA_without0(관대)
  - 99개의 문장 테스트
  - 측정 방법은 위와 동일 하지만 target label이 [0,0,0] 인 애들은 제외했음(모두 0인 애들)
- SA(엄격)
  - target label과 pred label이 완전히 일치해야 맞췄다고 봄.

In [ ]:
# SA_관대 
a2=accuracy(ours_7b,prompt = False,without0 = False, mercy = True)
b2=accuracy(ours_13b,prompt = False,without0 = False, mercy = True)
c2=accuracy(prompt_chat_7b,prompt =True,without0 = False, mercy = True)
d2=accuracy(prompt_7b,prompt =True,without0 = False, mercy = True)
e2=accuracy(acdp_7b,prompt = False,without0 = False, mercy = True)
f2=accuracy(acdp_13b,prompt = False,without0 = False, mercy = True)


In [17]:
# SA_without0_엄격 
a3=accuracy(ours_7b,prompt = False,without0 = True, mercy = False)
b3=accuracy(ours_13b,prompt = False,without0 = True, mercy = False)
c3=accuracy(prompt_chat_7b,prompt =True,without0 = True, mercy = False)
d3=accuracy(prompt_7b,prompt =True,without0 = True, mercy = False)
e3=accuracy(acdp_7b,prompt = False,without0 = True, mercy = False)
f3=accuracy(acdp_13b,prompt = False,without0 = True, mercy = False)


99


testl:   0%|          | 0/5 [00:00<?, ?it/s]

0.6666666666666666
99


testl:   0%|          | 0/5 [00:00<?, ?it/s]

0.6060606060606061


...

0.25471698113207547
99


testl:   0%|          | 0/5 [00:00<?, ?it/s]

0.6767676767676768
99


testl:   0%|          | 0/5 [00:00<?, ?it/s]

0.6464646464646465


In [20]:
# SA__without0_관대 
a4=accuracy(ours_7b,prompt = False,without0 = True, mercy = True)
b4=accuracy(ours_13b,prompt = False,without0 = True, mercy = True)
c4=accuracy(prompt_chat_7b,prompt =True,without0 = True, mercy = True)
d4=accuracy(prompt_7b,prompt =True,without0 = True, mercy = True)
e4=accuracy(acdp_7b,prompt = False,without0 = True, mercy = True)
f4=accuracy(acdp_13b,prompt = False,without0 = True, mercy = True)


99


testl:   0%|          | 0/5 [00:00<?, ?it/s]

0.7575757575757576
99


testl:   0%|          | 0/5 [00:00<?, ?it/s]

0.7272727272727273


...

0.3113207547169811
99


testl:   0%|          | 0/5 [00:00<?, ?it/s]

0.696969696969697
99


testl:   0%|          | 0/5 [00:00<?, ?it/s]

0.6767676767676768


In [21]:
print(a2,b2,c2,d2,e2,f2)

0.9314285714285714 0.9228571428571428 0.8722222222222222 0.8156424581005587 0.9142857142857143 0.9085714285714286


In [8]:
with open("/home/wooseok/llama2-chat/llama2-generated/result_generated_llama.pkl","rb") as f:
     prompt= pickle.load(f)
prompt_7b = prompt[['generated_utterance','attributes']]
prompt_7b.columns = ['hypothesis','symptom']
data = prompt_7b
data['label']= [np.array([0]*10)]*len(data)
Symptom_list = ['cough', 'fever', 'sore throat', 'breathing problem', 'headache', 'pain', 'virus' , 'eye problem' , 'allergy','pneumonia']

def set_label(row):
    label = [1 if symptom in row['symptom'] else 0 for symptom in Symptom_list]
    return label

# apply 함수를 사용하여 label 값을 설정
data['label'] = data.apply(set_label, axis=1)
del data['symptom']

data= data.reset_index(drop=True)

for i in range(len(data)):
    if np.sum(data['label'][i])==0 : 
        data.drop(i,inplace = True)
data= data.reset_index(drop=True)


'''
for i in range(len(data)):
    if data['hypothesis'][i] == '':
        data.drop(i,inplace = True)
data= data.reset_index(drop=True)
'''
print(len(data))
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
data


/tmp/ipykernel_10920/2117796263.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label']= [np.array([0]*10)]*len(data)
/tmp/ipykernel_10920/2117796263.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data.apply(set_label, axis=1)


106


/tmp/ipykernel_10920/2117796263.py:35: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,hypothesis,label
0,I m feeling so headache since morning.,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,"I was going to the hospital, but now my throat hurts. I think it's the coronavirus.","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
2,"Doe, I feel as if I have the cold and cough. I feel like vomiting, and also my nose is blocked. My skin is sweating, and I feel as if someone is hitting me with a hammer, especially on my head.","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,It is my sneeze that I am worried about. I keep coughing. Doctor:,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
4,It is very difficult to get up in the morning.,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0]"
5,thank,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
6,My symptoms are similar to those caused by a common cold. I have a fever and a dry cough. I have a sore throat.,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
7,yes yes.,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
8,okay thanks for all your help. doctor. doctor: you re welcome. thank you for visiting my clinic. Patient: okay thanks for being so understanding doctor.,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,"I have a cough, sir. Doctor:","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [9]:
test_set = CustomDataset(data, tokenizer, 256)
test_loader = DataLoader(test_set,batch_size=20, shuffle = False)



val_targets=[]
val_outputs=[]

with torch.no_grad():
    for batch_idx, data in enumerate(tqdm(test_loader,desc='testl')):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        val_targets.extend(targets.cpu().detach().numpy().tolist())
        val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    val_preds = (np.array(val_outputs) > 0.5).astype(int)
    val_targets = np.array(val_targets).astype(int)
    acc =0
    for i in range(len(val_preds)):
        if np.all(val_preds[i][np.where(val_targets[i]==1)]==1) == True: 
            acc+=1
    print("------------------------------",acc)
    print(acc/len(val_preds))
    print(classification_report(
        val_targets,
        val_outputs,
        output_dict=False,
        target_names=['A', 'B', 'C', 'D', 'E','G','H','I','J','K']
        ))


testl:   0%|          | 0/6 [00:00<?, ?it/s]

------------------------------ 33
0.3113207547169811


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [10]:
val_targets

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [11]:

with open("/home/wooseok/llama2-chat/llama2-chat-generated/result_generated.pkl","rb") as f:
     prompt_chat= pickle.load(f)
prompt_chat_7b = prompt_chat[['generated_utterance','attributes']]
prompt_chat_7b.columns = ['hypothesis','symptom']

data = prompt_chat_7b

data['label']= [np.array([0]*10)]*len(data)


Symptom_list = ['cough', 'fever', 'sore throat', 'breathing problem', 'headache', 'pain', 'virus' , 'eye problem' , 'allergy','pneumonia']

def set_label(row):
    label = [1 if symptom in row['symptom'] else 0 for symptom in Symptom_list]
    return label

# apply 함수를 사용하여 label 값을 설정
data['label'] = data.apply(set_label, axis=1)
del data['symptom']

data= data.reset_index(drop=True)
'''
for i in range(len(data)):
    if np.sum(data['label'][i])==0 : 
        data.drop(i,inplace = True)
data= data.reset_index(drop=True)
'''
for i in range(len(data)):
    if data['hypothesis'][i] == '':
        data.drop(i,inplace = True)
data= data.reset_index(drop=True)

data        

/tmp/ipykernel_4546/3080411673.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label']= [np.array([0]*10)]*len(data)
/tmp/ipykernel_4546/3080411673.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data.apply(set_label, axis=1)


,hypothesis,label
0,I've been experiencing this awful headache and...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,*sigh* I thought I was getting better but thi...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"Okay, I'll call. Thank you, Dr. What are some...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,My sore throat has been bothering me for days ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,"Well, I also have a runny nose since yesterda...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
356,"*No, I have only taken over the counter medic...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
357,My concerned is that I've been experiencing s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
358,So I'm cured then?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
359,Yes I went to the Hospital they did a chest X...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [12]:

test_set = CustomDataset(data, tokenizer, 256)
test_loader = DataLoader(test_set,batch_size=20, shuffle = False)



val_targets=[]
val_outputs=[]

with torch.no_grad():
    for batch_idx, data in enumerate(tqdm(test_loader,desc='testl')):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        val_targets.extend(targets.cpu().detach().numpy().tolist())
        val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    val_preds = (np.array(val_outputs) > 0.5).astype(int)
    val_targets = np.array(val_targets).astype(int)
    acc =0
    for i in range(len(val_preds)):
        if np.all(val_preds[i][np.where(val_targets[i]==1)]==1) == True: 
            acc+=1
    print("------------------------------",acc)
    print(acc/len(val_preds))

    #accc = metrics.accuracy_score(val_targets, val_preds)
    #print(accc)


testl:   0%|          | 0/19 [00:00<?, ?it/s]

------------------------------ 315
0.8725761772853186


In [10]:
data['hypothesis'][1] == ''

KeyError: 'hypothesis'

In [16]:
val_preds

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
val_targets = np.array(val_targets).astype(int)
val_targets

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
val_preds[0] == val_targets[0]

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True])

In [51]:
i =27
val_targets[i]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [52]:
val_preds[i]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [53]:
np.where(val_targets[i]==1)

(array([0]),)

In [54]:
val_preds[i][np.where(val_targets[i]==1)]

array([0])

In [55]:
np.all(val_preds[i][np.where(val_targets[i]==1)]==1)

False

In [108]:
acc =0
for i in range(len(val_preds)):
    if np.all(val_preds[i][np.where(val_targets[i]==1)]==1) == True: 
        acc+=1

print(acc/len(val_preds))

54

In [110]:
len(val_preds)

106

In [34]:
val_targets[i] - val_preds[i]

array([-1., -1.,  0., -1.,  0., -1.,  0.,  0.,  0.,  1.])

In [39]:
np.where(val_targets[i] == 1.0)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (1365436690.py, line 1)

In [ ]:

val_targets[i]

In [50]:
a = np.array(val_targets).astype(int)[]
a

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [51]:
np.where(a==1)

(array([9]),)

In [ ]:
for i in range(len(val_preds)):
    

In [1]:
import numpy as np
target_label = [[1,1,0,0,0],[1,1,0,1,1],[0,1,0,1,0],[0,1,0,0,0],[1,1,0,0,1]]
pred_label = [[1,1,1,0,0],[1,1,1,1,1],[0,1,0,1,0],[0,1,0,0,0],[1,1,0,0,1]]
accuracy = sum([np.array_equal(target, pred) for target, pred in zip(target_label, pred_label)]) / len(target_label)


In [2]:
accuracy

0.6

In [3]:
precision = sum([target[i] == 1 and pred[i] == 1 for i in range(len(target_label))]) / sum([pred[i] == 1 for i in range(len(target_label))])
precision

NameError: name 'target' is not defined

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

target_label = [[0,1,0,0,1],[1,1,0,1,1],[0,1,0,1,0],[0,1,0,1,0],[1,1,0,0,1]]
pred_label = [[1,1,1,0,0],[1,1,1,1,1],[0,1,0,1,0],[0,1,0,0,0],[1,1,0,0,1]]

accuracy = accuracy_score(target_label, pred_label)
precision = precision_score(target_label, pred_label, average='micro')
recall = recall_score(target_label, pred_label, average='micro')
f1 = f1_score(target_label, pred_label, average='micro')
print(accuracy, precision,recall,f1)

0.4 0.7857142857142857 0.8461538461538461 0.8148148148148148


In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 

target_label = [[0,1,0,0,1],[1,1,0,1,1],[0,1,0,1,0],[0,1,0,1,0],[1,1,0,0,1]]
pred_label = [[1,1,1,0,0],[1,1,1,1,1],[0,1,0,1,0],[0,1,0,0,0],[1,1,0,0,1]]

accuracy = accuracy_score(target_label, pred_label)
precision = precision_score(target_label, pred_label, average='macro')
recall = recall_score(target_label, pred_label, average='macro')
f1 = f1_score(target_label, pred_label, average='macro')
print(accuracy, precision,recall,f1)

0.4 0.7333333333333333 0.6666666666666666 0.68


/home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
print(classification_report(
    target_label,
    pred_label,
    output_dict=False,
    target_names=['A', 'B', 'C', 'D', 'E','G','H','I','J','K']
))

              precision    recall  f1-score   support

     class A       0.67      1.00      0.80         2
     class B       1.00      1.00      1.00         5
     class C       0.00      0.00      0.00         0
     class d       1.00      0.67      0.80         3
     class e       1.00      0.67      0.80         3

   micro avg       0.79      0.85      0.81        13
   macro avg       0.73      0.67      0.68        13
weighted avg       0.95      0.85      0.88        13
 samples avg       0.83      0.80      0.79        13



/home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
3.4/5

0.6799999999999999

In [10]:
#recall
#a 3/3

#b 5/5
#c 0
#d 1
#e

1.0

In [11]:
f1

0.923076923076923

FileNotFoundError: [Errno 2] No such file or directory: 'data1.csv'